In [1]:
from PyQt5.QtWidgets import QFileDialog, QApplication
import os
import os.path as ospath
import numpy as np
import pandas as pd

from picasso.picasso import io


def gui_fname(dir=None):
    """
    Select a file via a dialog and return the file name.
    """
    if dir is None: 
        dir ="./"

    app = QApplication([dir])
    fname = QFileDialog.getExistingDirectory(None, "Select a folder...", 
            dir)
    if isinstance(fname, tuple):
        return fname[0]
    else: 
        return str(fname)
    
def load_ring_data_df(dirname, filename):
    file = ospath.join(dirname, filename)

    try: 
        df = pd.read_pickle(file)
    except FileNotFoundError:
        print("No results of previously analyzed datasets were detected.")
        return None
    else: 
        print("Results of previously analyzed datasets were detected.")
        return df

def load_files(dirname):
    
    os.chdir(dirname)
    files = glob.glob("*.hdf5")
    
    if files:
        print("{} HDF5 files found.".format(len(files)))
    else:
        print("No HDF5 files found at: {}".format(dirname))
            
    return files
    
def load_data(path):

    try:
        locs, info = io.load_locs(path)
    except io.NoMetadataFileError:
        return None, None, None
    
    try:
        pixelsize = info[1]["Pixelsize"]
    except:  
        print("No pixelsize found in yaml file. Default 130 nm used.")

        
    return locs, info, pixelsize

def load_rings_by_radius(df_rings_data, path_rings, bin):
    
    ring_locs = []
    counter = 0
    for filename in df_rings_data['ring_filename']:
        locs_i, info_i, pixelsize = load_data(os.path.join(path_rings, filename))
        df_locs_i = pd.DataFrame.from_records(locs_i)
        df_locs_i['pick'] = df_locs_i['group']
        df_locs_i['group'] = np.full(len(df_locs_i), counter)
        counter +=1
        ring_locs.append(df_locs_i)
    
    df_ring_locs = pd.concat(ring_locs)
    
    info = info_i + [{'Ring radius interval': bin, 'Merged ring files': list(df_rings_data['ring_filename'])}]
    
    return df_ring_locs, info
    
    
    
    
def unfold_groups_square(locs, info, n_square = 100, spacing = 2):
    """
    Shifts grouped locs onto a rectangular grid of chosen length.
    Useful for locs that were processed with Picasso: Average.
    """


    if hasattr(locs, "group"):

        locs.x += (
            np.mod(locs.group, n_square) 
            * spacing
        )
        locs.y += (
            np.floor(locs.group / n_square) 
            * spacing
        )

        mean_x = np.mean(locs.x)
        mean_y = np.mean(locs.y)

        locs.x -= mean_x
        locs.y -= mean_y

        offset_x = np.absolute(np.min(locs.x))
        offset_y = np.absolute(np.min(locs.y))

        locs.x += offset_x + spacing
        locs.y += offset_y + spacing


    # Update width information
    info[0]["Height"] = int(np.ceil(np.max(locs.y))+spacing)
    info[0]["Width"] = int(np.ceil(np.max(locs.x))+spacing)
    
    return locs, info


def save_rings_locs(locs, info, path, cell_type, bin, filename = ''):
    

    ending = ".hdf5"    
    if filename != '':
        ending = filename + ending
    
    locs_name = "rings_{}_bin_{}_{}{}".format(cell_type, bin[0], bin[1], ending)
    locs_path_name = os.path.join(path, locs_name)
    
    if not os.path.isdir(path):
        os.makedirs(path)
    
    io.save_locs(locs_path_name, locs, info)

In [2]:
#path = gui_fname()
path = r'W:\users\reinhardt\z.software\Git\spor-PAINT\dev_sr\spor-paint\FtsZ\FtsZ_picks_all'

orientation = 'xy' # 'xy' or 'yz'
#filenames = load_files(path)

df_ring_data = load_ring_data_df(path, "ring_data_filter.pkl")

print(df_ring_data.keys())

radius_step = 15 # nm
sub_step = 5


radius_step = 100 # nm
sub_step = 25

Results of previously analyzed datasets were detected.
Index(['fov_id', 'cell_type', 'filename', 'group', 'residual', 'radius',
       'angle_between_ring_and_coverslip', 'Sigma', 'n_events', 'mean_bright',
       'mean_dark', 'fit_bright', 'fit_dark', 'filter_passed'],
      dtype='object')


In [3]:
"""
radius groups: resolution
rolling average

NeNAs:
5.9
5.82
4.37
3.97
4.16
9.1
"""

'\nradius groups: resolution\nrolling average\n\nNeNAs:\n5.9\n5.82\n4.37\n3.97\n4.16\n9.1\n'

In [4]:
min_radius_bin = df_ring_data['radius'].min()
print(min_radius_bin)

min_radius_bin = int(min_radius_bin / sub_step) * sub_step
print(min_radius_bin)

max_radius_bin = df_ring_data['radius'].max()
print(max_radius_bin)
max_radius_bin = (int(max_radius_bin / sub_step)+1) * sub_step
print(max_radius_bin)

bins = []
bins.append([min_radius_bin, min_radius_bin+radius_step])
i = 0
bin_end = bins[0][1]
while bin_end < max_radius_bin:
    
    bin_start = bins[i][0]+sub_step
    bin_end = bins[i][1]+sub_step
    bins.append([bin_start, bin_end])
    i+=1
    
print(bins)


79.17877960205078
75
36279.65234375
36300
[[75, 175], [100, 200], [125, 225], [150, 250], [175, 275], [200, 300], [225, 325], [250, 350], [275, 375], [300, 400], [325, 425], [350, 450], [375, 475], [400, 500], [425, 525], [450, 550], [475, 575], [500, 600], [525, 625], [550, 650], [575, 675], [600, 700], [625, 725], [650, 750], [675, 775], [700, 800], [725, 825], [750, 850], [775, 875], [800, 900], [825, 925], [850, 950], [875, 975], [900, 1000], [925, 1025], [950, 1050], [975, 1075], [1000, 1100], [1025, 1125], [1050, 1150], [1075, 1175], [1100, 1200], [1125, 1225], [1150, 1250], [1175, 1275], [1200, 1300], [1225, 1325], [1250, 1350], [1275, 1375], [1300, 1400], [1325, 1425], [1350, 1450], [1375, 1475], [1400, 1500], [1425, 1525], [1450, 1550], [1475, 1575], [1500, 1600], [1525, 1625], [1550, 1650], [1575, 1675], [1600, 1700], [1625, 1725], [1650, 1750], [1675, 1775], [1700, 1800], [1725, 1825], [1750, 1850], [1775, 1875], [1800, 1900], [1825, 1925], [1850, 1950], [1875, 1975], [1900,

In [5]:
"""
grid pattern

load rotated locs

move to grid positions

test version with horizontal ring

"""

for bin in bins:
    bin_start = bin[0]
    bin_end = bin[1]
    
    for cell_type in ['spor', 'veg']:
        print(bin, cell_type)
        df_rings = df_ring_data.copy()
        df_rings = df_rings[(df_rings['radius']>=bin_start) & (df_rings['radius']<bin_end) & (df_rings['filter_passed']=='Yes')]
        
        #print(df_rings['cell_type'])
        df_rings = df_rings[df_rings['cell_type']==cell_type]
        #print(df_rings['cell_type'])


        if len(df_rings) < 5:
            continue

        # Get filename of tilt corrected ring locs
        df_rings['ring_filename'] = 'fov_' + df_rings['fov_id'].astype(str) +'_' + df_rings['cell_type'].astype(str) +'_pick_' + df_rings['group'].astype(str) +'_ring_' + '0' +'_rot_' + orientation + '_update.hdf5'


        # Load all rings in df_rings, merge to one dataframe and assign group ID to every

        # All rings in this intervall will be assigned a new group ID (in the group column).
        # In order to trace back, which group ID (pick ID) they had before in the original FOV file, 
        # we will save the original group ID in a column called pick. 
        # both in the ring_data file as well as the generated merged hdf5. 

        df_rings['pick'] = df_rings['group']
        df_rings['group'] = np.arange(len(df_rings))
        
        path_average = os.path.join(path, 'analysis', 'average_' + orientation)
        if not os.path.isdir(path_average):
            os.makedirs(path_average)
            
        print(len(df_rings))
        
        # Save ring data
        df_rings.to_csv(os.path.join(path_average, 'rings_data_{}_bin_{}_{}{}'.format(cell_type, bin[0], bin[1], '.csv')))     
        df_rings.to_pickle(os.path.join(path_average, 'rings_data_{}_bin_{}_{}{}'.format(cell_type, bin[0], bin[1], '.pkl')))     


        path_rings = os.path.join(path, 'analysis', 'ring_locs')
        df_locs_merge, info_merge = load_rings_by_radius(df_rings, path_rings, bin)
        
        locs_merge = pd.DataFrame.to_records(df_locs_merge)
        save_rings_locs(locs_merge, info_merge, path_average, cell_type, bin, filename = '_overlay')

        # Unfold rings in a grid pattern
        locs_unfold, info_unfold = unfold_groups_square(locs_merge, info_merge, n_square = int(np.sqrt(len(df_rings))+1), spacing = 10)
    
        save_rings_locs(locs_unfold, info_unfold, path_average, cell_type, bin, filename = '_unfold')

    
        
    
    
    

[75, 175] spor
69
[75, 175] veg
17
[100, 200] spor
95
[100, 200] veg
23
[125, 225] spor
116
[125, 225] veg
27
[150, 250] spor
120
[150, 250] veg
32
[175, 275] spor
119
[175, 275] veg
37
[200, 300] spor
119
[200, 300] veg
40
[225, 325] spor
115
[225, 325] veg
41
[250, 350] spor
113
[250, 350] veg
42
[275, 375] spor
95
[275, 375] veg
32
[300, 400] spor
66
[300, 400] veg
24
[325, 425] spor
40
[325, 425] veg
16
[350, 450] spor
14
[350, 450] veg
5
[375, 475] spor
[375, 475] veg
[400, 500] spor
[400, 500] veg
[425, 525] spor
[425, 525] veg
[450, 550] spor
[450, 550] veg
[475, 575] spor
[475, 575] veg
[500, 600] spor
[500, 600] veg
[525, 625] spor
[525, 625] veg
[550, 650] spor
[550, 650] veg
[575, 675] spor
[575, 675] veg
[600, 700] spor
[600, 700] veg
[625, 725] spor
[625, 725] veg
[650, 750] spor
[650, 750] veg
[675, 775] spor
[675, 775] veg
[700, 800] spor
[700, 800] veg
[725, 825] spor
[725, 825] veg
[750, 850] spor
[750, 850] veg
[775, 875] spor
[775, 875] veg
[800, 900] spor
[800, 900]

[6500, 6600] spor
[6500, 6600] veg
[6525, 6625] spor
[6525, 6625] veg
[6550, 6650] spor
[6550, 6650] veg
[6575, 6675] spor
[6575, 6675] veg
[6600, 6700] spor
[6600, 6700] veg
[6625, 6725] spor
[6625, 6725] veg
[6650, 6750] spor
[6650, 6750] veg
[6675, 6775] spor
[6675, 6775] veg
[6700, 6800] spor
[6700, 6800] veg
[6725, 6825] spor
[6725, 6825] veg
[6750, 6850] spor
[6750, 6850] veg
[6775, 6875] spor
[6775, 6875] veg
[6800, 6900] spor
[6800, 6900] veg
[6825, 6925] spor
[6825, 6925] veg
[6850, 6950] spor
[6850, 6950] veg
[6875, 6975] spor
[6875, 6975] veg
[6900, 7000] spor
[6900, 7000] veg
[6925, 7025] spor
[6925, 7025] veg
[6950, 7050] spor
[6950, 7050] veg
[6975, 7075] spor
[6975, 7075] veg
[7000, 7100] spor
[7000, 7100] veg
[7025, 7125] spor
[7025, 7125] veg
[7050, 7150] spor
[7050, 7150] veg
[7075, 7175] spor
[7075, 7175] veg
[7100, 7200] spor
[7100, 7200] veg
[7125, 7225] spor
[7125, 7225] veg
[7150, 7250] spor
[7150, 7250] veg
[7175, 7275] spor
[7175, 7275] veg
[7200, 7300] spor
[7

[12725, 12825] spor
[12725, 12825] veg
[12750, 12850] spor
[12750, 12850] veg
[12775, 12875] spor
[12775, 12875] veg
[12800, 12900] spor
[12800, 12900] veg
[12825, 12925] spor
[12825, 12925] veg
[12850, 12950] spor
[12850, 12950] veg
[12875, 12975] spor
[12875, 12975] veg
[12900, 13000] spor
[12900, 13000] veg
[12925, 13025] spor
[12925, 13025] veg
[12950, 13050] spor
[12950, 13050] veg
[12975, 13075] spor
[12975, 13075] veg
[13000, 13100] spor
[13000, 13100] veg
[13025, 13125] spor
[13025, 13125] veg
[13050, 13150] spor
[13050, 13150] veg
[13075, 13175] spor
[13075, 13175] veg
[13100, 13200] spor
[13100, 13200] veg
[13125, 13225] spor
[13125, 13225] veg
[13150, 13250] spor
[13150, 13250] veg
[13175, 13275] spor
[13175, 13275] veg
[13200, 13300] spor
[13200, 13300] veg
[13225, 13325] spor
[13225, 13325] veg
[13250, 13350] spor
[13250, 13350] veg
[13275, 13375] spor
[13275, 13375] veg
[13300, 13400] spor
[13300, 13400] veg
[13325, 13425] spor
[13325, 13425] veg
[13350, 13450] spor
[1335

[18775, 18875] spor
[18775, 18875] veg
[18800, 18900] spor
[18800, 18900] veg
[18825, 18925] spor
[18825, 18925] veg
[18850, 18950] spor
[18850, 18950] veg
[18875, 18975] spor
[18875, 18975] veg
[18900, 19000] spor
[18900, 19000] veg
[18925, 19025] spor
[18925, 19025] veg
[18950, 19050] spor
[18950, 19050] veg
[18975, 19075] spor
[18975, 19075] veg
[19000, 19100] spor
[19000, 19100] veg
[19025, 19125] spor
[19025, 19125] veg
[19050, 19150] spor
[19050, 19150] veg
[19075, 19175] spor
[19075, 19175] veg
[19100, 19200] spor
[19100, 19200] veg
[19125, 19225] spor
[19125, 19225] veg
[19150, 19250] spor
[19150, 19250] veg
[19175, 19275] spor
[19175, 19275] veg
[19200, 19300] spor
[19200, 19300] veg
[19225, 19325] spor
[19225, 19325] veg
[19250, 19350] spor
[19250, 19350] veg
[19275, 19375] spor
[19275, 19375] veg
[19300, 19400] spor
[19300, 19400] veg
[19325, 19425] spor
[19325, 19425] veg
[19350, 19450] spor
[19350, 19450] veg
[19375, 19475] spor
[19375, 19475] veg
[19400, 19500] spor
[1940

[24850, 24950] veg
[24875, 24975] spor
[24875, 24975] veg
[24900, 25000] spor
[24900, 25000] veg
[24925, 25025] spor
[24925, 25025] veg
[24950, 25050] spor
[24950, 25050] veg
[24975, 25075] spor
[24975, 25075] veg
[25000, 25100] spor
[25000, 25100] veg
[25025, 25125] spor
[25025, 25125] veg
[25050, 25150] spor
[25050, 25150] veg
[25075, 25175] spor
[25075, 25175] veg
[25100, 25200] spor
[25100, 25200] veg
[25125, 25225] spor
[25125, 25225] veg
[25150, 25250] spor
[25150, 25250] veg
[25175, 25275] spor
[25175, 25275] veg
[25200, 25300] spor
[25200, 25300] veg
[25225, 25325] spor
[25225, 25325] veg
[25250, 25350] spor
[25250, 25350] veg
[25275, 25375] spor
[25275, 25375] veg
[25300, 25400] spor
[25300, 25400] veg
[25325, 25425] spor
[25325, 25425] veg
[25350, 25450] spor
[25350, 25450] veg
[25375, 25475] spor
[25375, 25475] veg
[25400, 25500] spor
[25400, 25500] veg
[25425, 25525] spor
[25425, 25525] veg
[25450, 25550] spor
[25450, 25550] veg
[25475, 25575] spor
[25475, 25575] veg
[25500

[30975, 31075] spor
[30975, 31075] veg
[31000, 31100] spor
[31000, 31100] veg
[31025, 31125] spor
[31025, 31125] veg
[31050, 31150] spor
[31050, 31150] veg
[31075, 31175] spor
[31075, 31175] veg
[31100, 31200] spor
[31100, 31200] veg
[31125, 31225] spor
[31125, 31225] veg
[31150, 31250] spor
[31150, 31250] veg
[31175, 31275] spor
[31175, 31275] veg
[31200, 31300] spor
[31200, 31300] veg
[31225, 31325] spor
[31225, 31325] veg
[31250, 31350] spor
[31250, 31350] veg
[31275, 31375] spor
[31275, 31375] veg
[31300, 31400] spor
[31300, 31400] veg
[31325, 31425] spor
[31325, 31425] veg
[31350, 31450] spor
[31350, 31450] veg
[31375, 31475] spor
[31375, 31475] veg
[31400, 31500] spor
[31400, 31500] veg
[31425, 31525] spor
[31425, 31525] veg
[31450, 31550] spor
[31450, 31550] veg
[31475, 31575] spor
[31475, 31575] veg
[31500, 31600] spor
[31500, 31600] veg
[31525, 31625] spor
[31525, 31625] veg
[31550, 31650] spor
[31550, 31650] veg
[31575, 31675] spor
[31575, 31675] veg
[31600, 31700] spor
[3160